In [ ]:
# !nvidia-smi
!git clone https://github.com/SWivid/F5-TTS.git
%cd F5-TTS

Cloning into 'F5-TTS'...
remote: Enumerating objects: 2783, done.
remote: Counting objects: 100% (992/992), done.
remote: Compressing objects: 100% (288/288), done.
remote: Total 2783 (delta 826), reused 707 (delta 704), pack-reused 1791 (from 3)
Receiving objects: 100% (2783/2783), 1.98 MiB | 6.84 MiB/s, done.
Resolving deltas: 100% (1724/1724), done.
/content/F5-TTS


In [ ]:
!pip install ffmpeg-python

In [ ]:
# !pip install -r requirements.txt
!pip install -e .

Obtaining file:///content/F5-TTS
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of cached-path to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of cached-path to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtim

In [ ]:
!wget https://huggingface.co/SWivid/F5-TTS/resolve/main/F5TTS_Base/model_1200000.pt -P ckpts/F5TTS_Base

--2025-03-19 04:41:23--  https://huggingface.co/SWivid/F5-TTS/resolve/main/F5TTS_Base/model_1200000.pt
Resolving huggingface.co (huggingface.co)... 13.35.202.40, 13.35.202.121, 13.35.202.34, ...
Connecting to huggingface.co (huggingface.co)|13.35.202.40|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/3f/a8/3fa80d46f55463b1ba65a7c105a934286e3c6c645cbe368ee7581a2bde17464b/c2f1bcbe1582a04468920abf227aa75f18faf57d24d5b141195eb4e55f39bc03?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model_1200000.pt%3B+filename%3D%22model_1200000.pt%22%3B&Expires=1742362884&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MjM2Mjg4NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzNmL2E4LzNmYTgwZDQ2ZjU1NDYzYjFiYTY1YTdjMTA1YTkzNDI4NmUzYzZjNjQ1Y2JlMzY4ZWU3NTgxYTJiZGUxNzQ2NGIvYzJmMWJjYmUxNTgyYTA0NDY4OTIwYWJmMjI3YWE3NWYxOGZhZjU3ZDI0ZDViMTQxMTk1ZWI0ZTU1ZjM5YmMwMz9yZXNwb25zZS1jb

In [ ]:
# Install pydub if not already installed
!pip install -q pydub

In [ ]:

from IPython.display import Javascript, display
from google.colab import output
from base64 import b64decode
from io import BytesIO
from pydub import AudioSegment

# Define JavaScript code for recording
RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time));
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader();
  reader.onloadend = e => resolve(e.srcElement.result);
  reader.readAsDataURL(blob);
});
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true });
  recorder = new MediaRecorder(stream);
  chunks = [];
  recorder.ondataavailable = e => chunks.push(e.data);
  recorder.start();
  await sleep(time);
  recorder.onstop = async () => {
    blob = new Blob(chunks);
    text = await b2text(blob);
    resolve(text);
  };
  recorder.stop();
});
"""

# Execute the JavaScript to define the record function
display(Javascript(RECORD))

# Function to record audio and save as WAV
def record_audio(seconds=5, filename="recorded_audio.wav"):
    audio_data = output.eval_js(f'record({seconds * 1000})')
    binary = b64decode(audio_data.split(',')[1])
    audio_segment = AudioSegment.from_file(BytesIO(binary), format="webm")
    audio_segment.export(filename, format="wav")
    print(f"Audio saved as {filename} in Colab filesystem!")
    return filename

# Record 5 seconds of audio
recorded_file = record_audio(seconds=10)

<IPython.core.display.Javascript object>

Audio saved as recorded_audio.wav in Colab filesystem!


In [ ]:
from google.colab import files
files.download('recorded_audio.wav')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
%pwd

'/content'

In [ ]:
!python /content/F5-TTS/src/f5_tts/infer/infer_cli.py \
    --model "F5TTS_v1_Base" \
    --ckpt_file "ckpts/F5TTS_Base/model_1200000.pt" \
    --ref_audio "reference.wav" \
    --ref_text "And I walk into this like sandwich shop I'm like hello mate you're right and then he's like what do you want I was like pickles please and this and that and blood with that and this and then and then like I was watching I was like you do know I'm speaking like with an English accent there and they're like really that's an English accent I'm like yes I said hello mate you're right" \
    --gen_text "hi from the smallest dot AI team"

2025-03-19 05:43:59.651275: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742363039.675058   18709 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742363039.682113   18709 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-19 05:43:59.706248: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Download Vocos from huggingface charactr/vocos-mel-24khz
Using F5TTS_v1_Base...

vocab :  /content/F5-TTS/src/f5_tts/infer/ex

In [ ]:
from IPython.display import Audio
Audio(data="/content/tests/infer_cli_basic.wav", rate=24000)

In [ ]:
sampling_freq = 44100
duration = 5

In [ ]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    mimeType : 'audio/webm;codecs=opus'
  };
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
    }
  };
  recorder.start();
};

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
recordButton.onclick = ()=>{
toggleRecording()
sleep(2000).then(() => {
  resolve(base64data.toString())
});
}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  # Convert the audio from webm to wav format using ffmpeg
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  # Save the wav file
  with open('tests/reference1.wav', 'wb') as f:
    f.write(output)

  # Optionally, read the saved audio for further processing
  sr, audio = wav_read(io.BytesIO(output))

  return audio, sr

In [ ]:
audio, sr = get_audio()

<ipython-input-33-6ab55334b860>:93: WavFileWarning: Reached EOF prematurely; finished at 109519 bytes, expected 4294967303 bytes from header.
  sr, audio = wav_read(io.BytesIO(output))


In [ ]:
!python  /content/F5-TTS/src/f5_tts/infer/infer_cli.py \
        --model "F5TTS_v1_Base" \
        --ckpt_file "ckpts/F5TTS_Base/model_1200000.pt" \
        --ref_audio "tests/reference1.wav" \
        --ref_text "" \
        --gen_text "So, this is F5 TTS, and it can match the flow of a speaker. How cool is that!" \
        -o "outputs"

2025-03-19 05:50:15.898331: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742363415.923377   20350 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742363415.930785   20350 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-19 05:50:15.961174: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Download Vocos from huggingface charactr/vocos-mel-24khz
Using F5TTS_v1_Base...

vocab :  /content/F5-TTS/src/f5_tts/infer/ex

In [ ]:
from IPython.display import Audio
Audio(data="/content/outputs/infer_cli_basic.wav", rate=24000)